In [20]:
import sys
import numpy as np
# from math import atan2, asin
from pathlib import Path
# from PIL import Image
# from PIL import ImageFont
# from PIL import ImageDraw 
np.set_printoptions(precision=4)

from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt

In [21]:
sys.path.append('/Users/jyao/JIAN/work/SAI/projects/modules')

In [22]:
from utils import SynJSON as SJ

In [23]:
dpath = Path('gazes_angles')

In [24]:
jfile = Path('gazes_angles/3.cam_singlecam.f_1.info.json')

In [25]:
js = SJ(jfile)
pitch, yaw = js.pitch_yaw(radian=False)
print(pitch, yaw)

14.929314613342285 -30.09840202331543


In [26]:
world2cam = js.world2cam()
world2cam

array([[ 1.000e+00,  0.000e+00,  0.000e+00, -1.786e-04],
       [ 0.000e+00,  1.000e+00,  0.000e+00, -1.610e+00],
       [ 0.000e+00,  0.000e+00,  1.000e+00, -8.715e-01],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  1.000e+00]])

In [27]:
gvectors = js.gaze_vectors()  #left and right
gvectors

(array([-0.4481,  0.2594,  0.8555]), array([-0.5179,  0.2588,  0.8153]))

In [28]:
def to_homogeneous(xs):
    xs_array = np.array(xs)
    assert (xs_array.shape[-1] == 3)
    ones_shape = list(xs_array.shape)
    ones_shape[-1] = 1
    homogeneous = np.concatenate([xs_array, np.ones(ones_shape)], axis=-1)
    return homogeneous
def from_homogeneous(xs):
    xs_array = np.array(xs)
    assert (xs_array.shape[-1] == 4)

    slice_but_last = list(xs_array.shape)
    slice_but_last[-1] = slice(None, -1)
    slice_but_last = tuple(slice_but_last)

    slice_last = list(xs_array.shape)
    slice_last[-1] = slice(-1, None)
    slice_last = tuple(slice_last)

    return xs_array[slice_but_last] / xs_array[slice_last]

In [29]:
gvecs = to_homogeneous(gvectors)
print(gvecs)

[[-0.4481  0.2594  0.8555  1.    ]
 [-0.5179  0.2588  0.8153  1.    ]]


In [30]:
cam2world = js.cam2world()
cam2world

array([[1.000e+00, 0.000e+00, 0.000e+00, 1.786e-04],
       [0.000e+00, 1.000e+00, 0.000e+00, 1.610e+00],
       [0.000e+00, 0.000e+00, 1.000e+00, 8.715e-01],
       [0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00]])

In [31]:
np.testing.assert_allclose(world2cam@cam2world, np.eye(4), atol=1e-10)

In [32]:

# def get_yaw_pitch_roll_world(item):
#     rotation = R.from_matrix(item[Modality.WORLD_TO_HEAD][:3,:3])

#     # We multiply resulting angles by [-1, 1, -1] to use the same positive axis
#     # as in job generaton json.
#     yaw, pitch, roll = rotation.as_euler("YXZ", degrees=True) * [-1, 1,-1]
#     return yaw, pitch, roll

In [33]:
r1 = R.from_matrix(world2cam[:3,:3])
cam_gaze_vectors = r1.apply(gvectors)
print(cam_gaze_vectors)
cam_gaze_vector = cam_gaze_vectors.mean(axis=0)
print(cam_gaze_vector)

[[-0.4481  0.2594  0.8555]
 [-0.5179  0.2588  0.8153]]
[-0.483   0.2591  0.8354]


In [34]:
std_vec=np.array([[0,0,1], [0,0,1]])
std_vec

array([[0, 0, 1],
       [0, 0, 1]])

In [35]:
r2 = R.align_vectors(cam_gaze_vectors, std_vec)

In [36]:
r2[0].apply([0,0,1])

array([-0.4834,  0.2593,  0.8361])

In [37]:
yaw, pitch, roll = r2[0].as_euler("YXZ", degrees=True)*[-1,-1,-1]
print(pitch, yaw)

15.029852179191643 30.033017381963916
